In [39]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import re

In [40]:
# Limpieza de texto
def limpiar_texto(texto):
    # Eliminar valores numéricos utilizando expresiones regulares
    texto_sin_numeros = re.sub(r'\d+', '', str(texto))

    # Remover tildes y caracteres especiales de entonacion
    texto_sin_tildes = unidecode(texto_sin_numeros)
    #texto_sin_tildes = unidecode(texto)

    # Quitar caracteres no alfabéticos y no espacios en blanco
    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto_sin_tildes)

    # Aplicar trim para eliminar espacios en blanco al principio y al final
    texto_limpio = texto_limpio.strip()
    
    return texto_limpio

In [41]:
## IMPORTE DE TODAS LAS TABLAS REQUERIDAS

#inpath = "E:/OneDrive/Datasets/Tuboleta/"
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/"

#A) TABLA dirty_cities
cities = pd.read_excel(inpath + 'dirty_cities.xlsx').\
    sort_values('MAIN_ADDR_TOWN', ascending = True)

#cities = cities['MAIN_ADDR_TOWN']

#################
# Descriptivos
print(cities.head(5))
print(' ')
print('Dimensiones:')
print(cities.shape)

print('----------------------------------------------------------')
#B) TABLA de codigos postales
codpost = pd.read_csv(inpath + 'codpost.csv', sep = '|',
                        # usecols=['%Cliente','Fecha', 'Producto'],
                        dtype={'Codigo Postal': 'object'})

#################
# Descriptivos
print(codpost.head(5))
print(' ')
print('Dimensiones:')
print(codpost.shape)


print('----------------------------------------------------------')
#C) TABLA de municipios de Colombia
dim_towns = pd.read_csv(inpath + 'dim_municipios.csv', sep = '|',
                        # usecols=['%Cliente','Fecha', 'Producto'],
                        dtype={'Codigo Postal': 'object'})

#################
# Descriptivos
print(dim_towns.head(5))
print(' ')
print('Dimensiones:')
print(dim_towns.shape)


print('----------------------------------------------------------')
#D) TABLA localidades de Bogota
loc_bog = pd.read_excel(inpath + 'localidades_bogota.xlsx')

#################
# Descriptivos
print(loc_bog.head(5))
print(' ')
print('Dimensiones:')
print(loc_bog.shape)

     MAIN_ADDR_TOWN  count
5            CAJICA      1
6            BOGOTA      1
7          GIRARDOT      1
8            BOGOTA      1
9            BOGOTA      4
 
Dimensiones:
(33185, 2)
----------------------------------------------------------
  Departamento Codigo Postal            Ciudad CodDep
0     AMAZONAS        915019  PUERTO SANTANDER     AM
1     AMAZONAS        917018        LA PEDRERA     AM
2     AMAZONAS        917017        LA PEDRERA     AM
3     AMAZONAS        917010        LA PEDRERA     AM
4     AMAZONAS        916058    MIRITI  PARANA     AM
 
Dimensiones:
(3681, 4)
----------------------------------------------------------
              REGION DEPARTAMENTO    MUNICIPIO  CAPITAL
0  Región Centro Sur     AMAZONAS      LETICIA  LETICIA
1  Región Centro Sur     AMAZONAS   EL ENCANTO  LETICIA
2  Región Centro Sur     AMAZONAS  LA CHORRERA  LETICIA
3  Región Centro Sur     AMAZONAS   LA PEDRERA  LETICIA
4  Región Centro Sur     AMAZONAS  LA VICTORIA  LETICIA
 
Dimensi

In [42]:
# left join
cities_ = pd.merge(cities, codpost[['Codigo Postal', 'Ciudad']],\
                          left_on='MAIN_ADDR_TOWN', right_on= codpost['Codigo Postal'].astype(str),\
                          how='left')

# Rellenar los valores nulos en 'Ciudad' con los valores de 'MAIN_ADDR_TOWN'
cities_['Ciudad'].fillna(cities_['MAIN_ADDR_TOWN'], inplace=True)

#################
# Descriptivos
print(cities_.head(5))
print(' ')
print('Dimensiones:')
print(cities_.shape)
print('Summary:')
codpost.info()
print(' ')
print('Descriptivos:')
print(cities_.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(cities_.isnull().sum())  # total de nulls por variable

     MAIN_ADDR_TOWN  count Codigo Postal            Ciudad
0            CAJICA      1           NaN            CAJICA
1            BOGOTA      1           NaN            BOGOTA
2          GIRARDOT      1           NaN          GIRARDOT
3            BOGOTA      1           NaN            BOGOTA
4            BOGOTA      4           NaN            BOGOTA
 
Dimensiones:
(33185, 4)
Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3681 entries, 0 to 3680
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Departamento   3681 non-null   object
 1   Codigo Postal  3681 non-null   object
 2   Ciudad         3681 non-null   object
 3   CodDep         3440 non-null   object
dtypes: object(4)
memory usage: 115.2+ KB
 
Descriptivos:
         count   mean     std  min  25%  50%  75%        max
count  33185.0  94.21  6773.8  1.0  1.0  1.0  2.0  1034485.0
 
Nulos por campo:
MAIN_ADDR_TOWN        4
count                 0


In [43]:
# Valores con codigo postal matched
prueba = cities_[cities_['Codigo Postal'].notnull()]

print(prueba['count'].sum())

#################
# Descriptivos
print(prueba.head(5))
print(' ')
print('Dimensiones:')
print(prueba.shape)

6415
     MAIN_ADDR_TOWN  count Codigo Postal            Ciudad
1698         110111    901        110111  BOGOTA   USAQUEN
1720         110121    137        110121  BOGOTA   USAQUEN
1727         110131     61        110131  BOGOTA   USAQUEN
1730         110141     40        110141  BOGOTA   USAQUEN
1735         110151     14        110151  BOGOTA   USAQUEN
 
Dimensiones:
(495, 4)


In [44]:
# Escritura
prueba.to_csv(inpath +'prueba.csv', 
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               sep = '|',
               decimal = '.')  

In [45]:
# Funcion like ampliada
def asignar_ciudad(row):
    condiciones = {
        'BOGO': 'BOGOTA',
        'LOCALID': 'BOGOTA',
        'MEDEL': 'MEDELLIN',
        'CALIFORNIA': 'CALIFORNIA USA',
        'MEXICALI': 'MEXICALI',
        'CALI': 'CALI',
        'VALLEDU': 'VALLEDUPAR',
        ' VALLE': 'CALI',
        'CARTAGE': 'CARTAGENA',
        'QUILLA': 'BARRANQUILLA',
        'BARRANQUI': 'BARRANQUILLA',
        'VILLAVI': 'VILLAVICENCIO',
        'YOPAL': 'YOPAL',
        'TUNJA': 'TUNJA',
        'CUCUT': 'CUCUTA',
        'MANIZA': 'MANIZALES',
        'IBAG': 'IBAGUE',
        'NEIV': 'NEIVA', 
        'SANTA MART': 'SANTA MARTA',
        'BUCARA': 'BUCARAMANGA',
        'MONTERI': 'MONTERIA',
        'POPAYA': 'POPAYAN',
        'ARMENI': 'ARMENIA',
        'PEREIR': 'PEREIRA',
        'UBATE': 'UBATE',
        'CHIA': 'CHIA',
        'SOLEDA': 'SOLEDAD',
        'BELLO': 'BELLO',
        # Puedes agregar más condiciones aquí según sea necesario
    }
    
    for condicion, ciudad in condiciones.items():
        if condicion in row['CIUDAD']:
            return ciudad
    
    return row['CIUDAD']



In [46]:
# def asignar_ciudad(row):
#     patrones = {
#         r'^POPAYA$': 'POPAYAN',
#         r'^ARMENI$': 'ARMENIA',
#         r'^PEREIR$': 'PEREIRA',
#         r'^UBATE$': 'UBATE',
#         # Puedes agregar más patrones aquí según sea necesario
#     }
    
#     for patron, ciudad in patrones.items():
#         if re.match(patron, row['Ciudad']):
#             return ciudad
    
#     return row['Ciudad']

In [47]:
total = sum(cities_['count'])
print('num registros: ', total)

dim_municipios_ = cities_.sort_values(by='count', ascending=False).\
       assign(Ciudad = lambda x: x['Ciudad'].apply(limpiar_texto),
              PARTICIPACION = lambda x: x['count']/total,
              TOTAL = lambda x: np.cumsum(x['PARTICIPACION'])).\
       rename(columns = {'count': 'COUNT'})
       #groupby('DEPARTAMENTO').agg({'COUNT': 'sum','PORCENTAJE': 'sum'}).reset_index().\


# left join con localidades
dim_municipios_1 = pd.merge(dim_municipios_, loc_bog,\
                          left_on='Ciudad', right_on= 'LOCALIDAD',\
                          how='left')

#dim_municipios_1 = dim_municipios_1[dim_municipios_1['CIUDAD'].notnull()]



# Rellenar los valores nulos en 'Ciudad' con los valores de 'MAIN_ADDR_TOWN'
dim_municipios_1['CIUDAD'].fillna(dim_municipios_1['Ciudad'], inplace=True)



# Aplicar la condición para buscar coincidencias parciales
dim_municipios_1['CIUDAD'] = dim_municipios_1.apply(asignar_ciudad, axis=1)

#dim_municipios_1 = dim_municipios_1[dim_municipios_1['Ciudad'] == 'FONTIBON']

#################
# Descriptivos
print(dim_municipios_1.head(5))
print(' ')
print('Dimensiones:')
print(dim_municipios_1.shape)

dim_municipios_1['CIUDAD'] = np.where(
    (dim_municipios_1['CIUDAD'] == 'BTA') |
    (dim_municipios_1['CIUDAD'] == 'BOGTA') |
    (dim_municipios_1['CIUDAD'] == 'BOG') |
    (dim_municipios_1['CIUDAD'] == 'BOOGTA') |
    (dim_municipios_1['CIUDAD'] == 'BGTA') |
    (dim_municipios_1['CIUDAD'] == 'BGOTA') |
    (dim_municipios_1['CIUDAD'] == 'BGA') |
    (dim_municipios_1['CIUDAD'] == 'CUNDINAMARCA')
    ,
    'BOGOTA',
    dim_municipios_1['CIUDAD']
)


dim_municipios_1['CIUDAD'] = np.where(
    (dim_municipios_1['CIUDAD'] == 'BAQ') |
    (dim_municipios_1['CIUDAD'] == 'BQLLA')  |
    (dim_municipios_1['CIUDAD'] == 'BQ')
    ,
    'BARRANQUILLA',
    dim_municipios_1['CIUDAD']
)



num registros:  3126459
  MAIN_ADDR_TOWN    COUNT Codigo Postal        Ciudad  PARTICIPACION  \
0            NaN  1034485           NaN           nan       0.330881   
1         BOGOTA   615709           NaN        BOGOTA       0.196935   
2    BOGOTÁ D.C.   173332           NaN     BOGOTA DC       0.055440   
3   BARRANQUILLA    92630           NaN  BARRANQUILLA       0.029628   
4       MEDELLIN    86092           NaN      MEDELLIN       0.027537   

      TOTAL LOCALIDAD        CIUDAD  
0  0.330881       NaN           nan  
1  0.527816       NaN        BOGOTA  
2  0.583256       NaN        BOGOTA  
3  0.612884       NaN  BARRANQUILLA  
4  0.640420       NaN      MEDELLIN  
 
Dimensiones:
(33185, 8)


In [48]:

prueba = dim_municipios_1[['CIUDAD','COUNT','PARTICIPACION']].\
    groupby('CIUDAD').agg({'COUNT': 'sum','PARTICIPACION': 'sum'}).\
        sort_values(by='PARTICIPACION', ascending=False).reset_index()

#################
# Descriptivos
#print(prueba.head(5))
print(' ')
print('Dimensiones:')
print(prueba.shape)


# Escritura
prueba.to_csv(inpath +'prueba.csv', 
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               sep = '|',
               decimal = '.')  


 
Dimensiones:
(17108, 3)


In [49]:
# left join para traer la region y el departamento
dim_municipios = pd.merge(dim_municipios_1, dim_towns,\
                          left_on='CIUDAD', right_on= 'MUNICIPIO',\
                          how='left')

dim_municipios['MUNICIPIO'] = np.where(
    (dim_municipios['CIUDAD'] == '') |
    (dim_municipios['CIUDAD'] == 'nan') |
    (dim_municipios['CIUDAD'].isnull()),
    'NO DISPONIBLE',
    dim_municipios['MUNICIPIO']
)

#################
# Descriptivos
print(dim_municipios.head(5))
print(' ')
print('Dimensiones:')
print(dim_municipios.shape)
print('Summary:')
dim_municipios.info()
print(' ')
print('Descriptivos:')
print(dim_municipios.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(dim_municipios.isnull().sum())  # total de nulls por variable

  MAIN_ADDR_TOWN    COUNT Codigo Postal        Ciudad  PARTICIPACION  \
0            NaN  1034485           NaN           nan       0.330881   
1         BOGOTA   615709           NaN        BOGOTA       0.196935   
2    BOGOTÁ D.C.   173332           NaN     BOGOTA DC       0.055440   
3   BARRANQUILLA    92630           NaN  BARRANQUILLA       0.029628   
4       MEDELLIN    86092           NaN      MEDELLIN       0.027537   

      TOTAL LOCALIDAD        CIUDAD                           REGION  \
0  0.330881       NaN           nan                              NaN   
1  0.527816       NaN        BOGOTA            Región Centro Oriente   
2  0.583256       NaN        BOGOTA            Región Centro Oriente   
3  0.612884       NaN  BARRANQUILLA                    Región Caribe   
4  0.640420       NaN      MEDELLIN  Región Eje Cafetero - Antioquia   

  DEPARTAMENTO      MUNICIPIO       CAPITAL  
0          NaN  NO DISPONIBLE           NaN  
1       BOGOTA         BOGOTA        BOGOT

In [50]:
# Valores con codigo postal matched
prueba = dim_municipios[dim_municipios['MUNICIPIO'].notnull()]


print('Registros: ', prueba['COUNT'].sum())
print('Percent: ', prueba['PARTICIPACION'].sum())

#################
# Descriptivos
print(prueba.head(5))
print(' ')
print('Dimensiones:')
print(prueba.shape)

Registros:  3077986
Percent:  0.9844958785642159
  MAIN_ADDR_TOWN    COUNT Codigo Postal        Ciudad  PARTICIPACION  \
0            NaN  1034485           NaN           nan       0.330881   
1         BOGOTA   615709           NaN        BOGOTA       0.196935   
2    BOGOTÁ D.C.   173332           NaN     BOGOTA DC       0.055440   
3   BARRANQUILLA    92630           NaN  BARRANQUILLA       0.029628   
4       MEDELLIN    86092           NaN      MEDELLIN       0.027537   

      TOTAL LOCALIDAD        CIUDAD                           REGION  \
0  0.330881       NaN           nan                              NaN   
1  0.527816       NaN        BOGOTA            Región Centro Oriente   
2  0.583256       NaN        BOGOTA            Región Centro Oriente   
3  0.612884       NaN  BARRANQUILLA                    Región Caribe   
4  0.640420       NaN      MEDELLIN  Región Eje Cafetero - Antioquia   

  DEPARTAMENTO      MUNICIPIO       CAPITAL  
0          NaN  NO DISPONIBLE          

In [51]:
dim_municipios = dim_municipios[['MAIN_ADDR_TOWN','MUNICIPIO','REGION','DEPARTAMENTO','CAPITAL','PAIS']]


# Escritura
dim_municipios.to_excel(inpath +'cleaned_cities.xlsx',
               index=False, # El argumento 'index=False' evita que se incluyan los índices en el archivo CSV
               #sep = '|',
               #decimal = '.'
               )